In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

box_DIR = "../data/box/"
img_DIR_P = "../data/images/players/"

In [ ]:
cols = ['PLAYER_ID', 'PLAYER_NAME', 'WL' , 'MIN', 'FGM',
       'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'PLUS_MINUS', ]
dfa = []
for year in range(1980,2024):
    df1 = pd.read_parquet(box_DIR + f"NBA_Box_P_Base_{year}.parquet",columns=cols)
    df1["W"] = df1["WL"] == "W"
    df1["L"] = df1["WL"] == "L"
    df1["W"] = np.where(df1["W"],1,0)
    df1["L"] = np.where(df1["L"],1,0)
    # df1 = df1.drop(columns=["WL"])
    df1 = df1.rename(columns={"WL":"GP"})
    df1["GP"] = 1
    df2 =df1.groupby(['PLAYER_ID', 'PLAYER_NAME']).sum()
    dfa.append(df2)
df3 = pd.concat(dfa)
df3 = df3
df4 =df3.groupby(['PLAYER_ID', 'PLAYER_NAME']).sum().reset_index()

In [ ]:
dfp = pd.read_csv("../data/" + "NBA_players_database.csv")
dfp.columns = ["PLAYER_ID","Name","Season","Last"]
dfp = dfp[["PLAYER_ID","Season"]]
df5 = pd.merge(df4,dfp,on=["PLAYER_ID"])
df5 = df5.drop(columns=['PLAYER_ID', 'PLAYER_NAME'])

In [ ]:
df6 = df5.groupby(["Season"]).sum().reset_index()
df6 = df6.query("Season > 1979")

In [ ]:
df6["S"] = df6["Season"].astype(str)
df6["S"] = df6["S"].str[-2:]

In [ ]:
df6 = df6.sort_values("PTS")
df6["Season"] = pd.Categorical(df6["Season"],categories=df6["Season"].to_list())

In [ ]:
p = (
    ggplot(df6,aes(x="Season",y="PTS"))
    + geom_bar(stat="identity", alpha=0.5)
    # + geom_text(aes(label = "S"),size=6,nudge_y=-2000)
    + theme_idv
    + coord_flip()
    + labs(
        title="Games Played by all Players in a Rookie Class",
        subtitle="Since 1980",
        y="Games Played",
        caption="@sradjoker | source:nba.com/stats"
    )
    + theme(
        figure_size=(7,10)
    )
)
p

In [ ]:
gfhgh

## Active Players Per Class

In [ ]:
cols = ['PLAYER_ID', 'PLAYER_NAME', 'WL' , 'MIN', 'FGM',
       'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'PLUS_MINUS', ]
year = 2023
df1 = pd.read_parquet(box_DIR + f"NBA_Box_P_Base_{year}.parquet",columns=cols)
df1["W"] = df1["WL"] == "W"
df1["L"] = df1["WL"] == "L"
df1["W"] = np.where(df1["W"],1,0)
df1["L"] = np.where(df1["L"],1,0)
# df1 = df1.drop(columns=["WL"])
df1 = df1.rename(columns={"WL":"GP"})
df1["GP"] = 1
df2 =df1.groupby(['PLAYER_ID', 'PLAYER_NAME']).sum().reset_index()
dfp = pd.read_csv("../data/" + "NBA_players_database.csv")
dfp.columns = ["PLAYER_ID","Name","Season","Last"]
dfp = dfp[["PLAYER_ID","Season"]]
df3 = pd.merge(df2,dfp,on=["PLAYER_ID"])

In [ ]:
df31 = df3.drop(columns=['PLAYER_ID', 'PLAYER_NAME'])
df4 = df31.groupby(["Season"]).count().reset_index()
df4["Players"] = df4["GP"]
cats = df4["Season"].to_list()
cats.reverse()
# df4["Season"] = pd.Categorical(df4["Season"],categories=cats)

In [ ]:
from plotnine import scale_x_reverse
p = (
    ggplot(df4,aes(x="Season",y="Players"))
    + geom_bar(stat="identity", alpha=0.5)
    # + geom_text(aes(label = "S"),size=6,nudge_y=-2000)
    + scale_x_reverse(breaks=range(2023,2002,-1))
    + geom_text(aes(label = "Players"),size=9,nudge_y=1)

    # + scale_x_continuous()
    + theme_idv
    + coord_flip()
    + labs(
        title="Active Players per Rookie Class",
        subtitle="Rookie Class: First Game Played in that Season",
        y="Active Players",
        caption="@sradjoker | source:nba.com/stats"
    )
    + theme(
        figure_size=(7,10)
    )
)
p

In [ ]:
df3g = df3.groupby("Season")
keys = list(df3g.groups)
dfb = []
for key in keys:
    df5 = df3g.get_group(key)
    df5 = df5.sort_values("MIN",ascending=False)
    df5["Count"] = 1
    df5["Count"] = df5["Count"].cumsum()
    dfb.append(df5)
df6 = pd.concat(dfb)
df6["Image"] = img_DIR_P + df6["PLAYER_ID"].astype(str) + ".png"
df6 = df6.query("Season < 2013")

In [ ]:
from plotnine import scale_x_reverse
p = (
    ggplot(df6,aes(x="Count",y="Season",image="Image"))
    + geom_point()
    + geom_image(size=0.18)
    # + geom_text(aes(label = "S"),size=6,nudge_y=-2000)
    + scale_y_reverse(breaks=range(2012,2002,-1))
    + theme_xkcd(base_size=14)
    # + coord_flip()
    + labs(
        title="Active Players per Rookie Class: 2003-2012",
        subtitle="Rookie Class: First Game Played in that Season | Sorted by Highest Minutes Played to Lowest",
        y="Active Players",
        caption="@sradjoker | source:nba.com/stats"
    )
    + theme(
        figure_size=(10,7),
        panel_spacing=1,
        plot_title=element_text(size=24),
    )
)
p

In [ ]:
asdasd

## Most stats since a season

In [ ]:
cols = ['PLAYER_ID', 'PLAYER_NAME', 'WL' , 'MIN', 'FGM',
       'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'PLUS_MINUS', ]
dfa = []
for year in range(2015,2024):
    df1 = pd.read_parquet(box_DIR + f"NBA_Box_P_Base_{year}.parquet",columns=cols)
    df1["W"] = df1["WL"] == "W"
    df1["L"] = df1["WL"] == "L"
    df1["W"] = np.where(df1["W"],1,0)
    df1["L"] = np.where(df1["L"],1,0)
    # df1 = df1.drop(columns=["WL"])
    df1 = df1.rename(columns={"WL":"GP"})
    df1["GP"] = 1
    df2 =df1.groupby(['PLAYER_ID', 'PLAYER_NAME']).sum()
    dfa.append(df2)
df3 = pd.concat(dfa)
df3 = df3
df4 =df3.groupby(['PLAYER_ID', 'PLAYER_NAME']).sum().reset_index()
df4["FG_PCT"] = round(df4["FGM"]/df4["FGA"],3)
df4["FG3_PCT"] = round(df4["FG3M"]/df4["FG3A"],3)
df4["FT_PCT"] = round(df4["FTM"]/df4["FTA"],3)

In [ ]:
df4.columns

In [ ]:
df4 = df4.sort_values("PTS",ascending=False).reset_index(drop=True)
df4.index +=1
df4.head(10)
# df4.query("PLAYER_NAME == 'Nikola Jokic'")